# How to grow the pie (in consulting)

Human organizations tend to move to higher levels of entropy unless there is a concerted effort to keep members of that organization working together. For a profesional consulting organization like d5.ai where members contribute their time and effort hoping that the collective action will bring higher returns that individual action in isolation.

In order to incentivice working towards the common goals of the organization, here we'll propose several mechanisms that will help on that.

## Use the relay mechanism for growing the pot

All the relay payments are paid to a wallet belonging to the DAO and the governance of the DAO is modified proportional to that deposit. So say that the initial pot size of the DAO is **0**, and governance tokens are evenly split. The state of the DAO will be (the array represents goverance tokens for Alex, Toni, Evgeny, Lars, Mats, Guy, Anders):

```
(0, [1000, 1000, 1000, 1000, 1000, 1000, 1000])
```

Then Alex relays a 10000 job to Toni. Alex gets the 1000 relay in DAI that goes to the DAO and Toni gets 9k$ in cash as a result of the job performed. The state of the DAO changes to:

```
(1000, [2000, 1000, 1000, 1000, 1000, 1000, 1000])
```

Ownership for Alex of d5.ai goes from $0.1428$ to $0.25$

Now d5.ai agrees on paying dividends to all members using the whole pot, Alex will get 250 DAI and all other members will get 125 DAI. Now the state of the DAO is 

```
(0, [2000, 1000, 1000, 1000, 1000, 1000, 1000])
```

Now it is Toni who finds a 10000 gig and relays to Alex. The state of the DAO changes to:

```
(1000, [2000, 2000, 1000, 1000, 1000, 1000, 1000])
```

Now we find that we are lagging on our Medium post publication rate and it is impacting our brand awareness. In the bounty session we assign Evegeny with a 500 tokens bounty for creating a new blog post. The DAO state changes to 

```
(1000, [2000, 2000, 1500, 1000, 1000, 1000, 1000])
```

We again split dividends and Toni and Alex get 2/9 of the pot, Evegeny 1/6, the rest of the team gets 1/9. Compared to what one would get by relaying if this mechanism wasn't in place the net result is:

- Alex:   -540 \$  (250 + 210 - 1000)
- Toni:   -665 \$  (125 + 210 - 1000) 
- Evgeny: +282 \$  (125 + 166)
- Rest:   +230 \$  (125 + 105)

Let me try to formalize some parameters in the scenario above:

- There is a 1 to 1 correspondence between DFC and DAI
- $C$ is the initial DFC capital in the DAO, in the example $C = 1000$
- $\alpha$ is the constant by which relay money to the DAO is reflected as an increase in DFC for the relayer user. In the example $\alpha = 1$

It is clear that this mechanism discourages contributing to the DAO because doing nothing is more beneficial than relaying unless internal tasks for the DAO (like Evgeny's in the example) bring more value than otherwise. 
So let's define a multiplier $k \geq 1$ that defines how DFC spent in the DAO brings relays that otherwise wouldn't happen.

Let's perform some simulations using different values for the constants. The first thing I thought of is the initial DFC assigned to each member in order to prevent a "dictator" to gain much power in the initial stages of the DAO but maybe a mechanism of `rage quitting` is enough. 

In [9]:
from collections import defaultdict

def split_pot(dao_state):
    dfcs = dao_state['dfcs']
    shared_pool = sum(dfcs.values())
    dividends = {member: (dao_state['dai_pool']*dfcs[member]/shared_pool) for member in dfcs}
    dao_state['dai_pool'] = 0
    return dividends

def relay_to_dao(who, amount, dao_state):
    dao_state['dai_pool'] += amount
    dao_state['dfcs'][who] += amount
    
def work_for_the_dao(who, amount, dao_state):
    dao_state['dfcs'][who] += amount

members = ['Alex', 'Toni', 'Evgeny', 'Lars', 'Mats', 'Guy', 'Anders']
for C in [0, 10, 100, 1000, 10000]:
    dfcs = {member:C for member in members}
    state = { 'dai_pool': 0, 'dfcs': dfcs}
    
    relay_to_dao('Alex', 1000, state)
    dividends_1 = split_pot(state)
    relay_to_dao('Toni', 1000, state)
    work_for_the_dao('Evgeny', 500, state)
    dividends_2 = split_pot(state)
    
    net_result = defaultdict(int)
    for dividend in [dividends_1, dividends_2]:
        for member, amount in dividend.items():
            net_result[member] += amount
            
    net_result['Alex'] -= 1000
    net_result['Toni'] -= 1000
    print("Net result for C={}".format(C))
    for member, amount in net_result.items():
        print("   {}: {}".format(member, amount))


Net result for C=0
   Alex: 400.0
   Toni: -600.0
   Evgeny: 200.0
   Lars: 0.0
   Mats: 0.0
   Guy: 0.0
   Anders: 0.0
Net result for C=10
   Alex: 336.9213425942762
   Toni: -597.6580966580602
   Evgeny: 207.78937415906034
   Lars: 13.236844976180953
   Mats: 13.236844976180953
   Guy: 13.236844976180953
   Anders: 13.236844976180953
Net result for C=100
   Alex: -9.191176470588289
   Toni: -597.4264705882354
   Evgeny: 246.3235294117647
   Lars: 90.0735294117647
   Mats: 90.0735294117647
   Guy: 90.0735294117647
   Anders: 90.0735294117647
Net result for C=1000
   Alex: -539.4736842105264
   Toni: -664.4736842105264
   Evgeny: 282.89473684210526
   Lars: 230.26315789473682
   Mats: 230.26315789473682
   Guy: 230.26315789473682
   Anders: 230.26315789473682
Net result for C=10000
   Alex: -693.3462846041768
   Toni: -707.4307916464303
   Evgeny: 285.67265662943174
   Lars: 278.77610490529383
   Mats: 278.77610490529383
   Guy: 278.77610490529383
   Anders: 278.77610490529383
